Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины
$X\sim N\left(\mu,\sigma^2\right)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.

In [6]:
import numpy as np
import pandas as pd
import scipy.stats as stats

round(stats.norm.ppf(1 - 0.003 / 2), 4)

2.9677

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [113]:
p_asp = 104. / 11037.
p_plc = 189. / 11034.

round(p_plc - p_asp, 4)

0.0077

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [198]:
from scipy import stats

def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

# Биномиальное распределение при приёме плацебо
plc_sample = np.hstack((np.ones(shape=189), np.zeros(shape=11034 - 189)))
np.random.shuffle(plc_sample)

# Биномиальное распределение при приёме аспирина
asp_sample = np.hstack((np.ones(shape=104), np.zeros(shape=11037 - 104)))
np.random.shuffle(asp_sample)

conf_interval = proportions_confint_diff_ind(plc_sample, asp_sample)
round(conf_interval[1], 4)

0.0107

Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X\sim Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$ Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как
$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174$ Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [60]:
odds_asp = p_asp / (1 - p_asp)
odds_plc = p_plc / (1 - p_plc)

round(odds_plc / odds_asp, 4)

1.8321

Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

* составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, потом все нули;
* установите random seed=0;
* сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [224]:
np.random.seed(0)

def get_chance(sample):
    return float(sum(sample) / (len(sample) - sum(sample)))

def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# Биномиальное распределение при приёме плацебо
plc_sample = np.hstack((np.ones(shape=189), np.zeros(shape=11034 - 189)))
# Биномиальное распределение при приёме аспирина
asp_sample = np.hstack((np.ones(shape=104), np.zeros(shape=11037 - 104)))

plc_chance = np.array(list(map(get_chance, get_bootstrap_samples(plc_sample, 1000))))
asp_chance = np.array(list(map(get_chance, get_bootstrap_samples(asp_sample, 1000))))

round(stat_intervals(plc_chance / asp_chance, 0.05)[0], 4)

1.4629

In [238]:
stats.t.sf(23.21, 99)

3.918634755068266e-42